# Covid-19 Detector 

In [1]:
# For deeplearning
import tensorflow as tf       
from tensorflow import keras 
from keras.utils.np_utils import to_categorical 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
# For managing files   
import os
# For image manipulation
from PIL import Image
import cv2
import imutils
# Vectorized code, translate images into vector arrays
import numpy as np
# Data processing, confusion matrix and classification report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report
# For visualization and image display
import matplotlib.pyplot as plt
import cv2
import imutils
from google.colab.patches import cv2_imshow
# Utilities
from collections import Counter
import copy
import random

: 

: 

In [ ]:
def dict_labels(path):
    '''
    Returns a dictionary with the name of the picture and the appropriate labels

    Arguments:
        path: Path to the file with the labels
    '''
    with open(path) as f:
        lines = f.readlines()
    split_lines = [line.split() for line in lines]

    return {split_line[1]:split_line[2] for split_line in split_lines}

def load_colorize_images(path, path_doc, image_size=(112,112)): 
    '''
    Returns two arrays: one with the pseudo-colorized images and another one with the output labels
    
    Arguments: 
        path: Path to the image
        path_doc: Path to the document with the labels
        image_size: Resized image to 112x112 as default
    '''

    data = []
    titles = []

    for file in os.listdir(path):
        path_file = path + '/' + file
        img = cv2.imread(path_file,0)
        img = cv2.resize(img, image_size)
        img = cv2.applyColorMap(img, cv2.COLORMAP_BONE)
        img = (img/255).astype(np.float32)
        data.append(img)
        titles.append(file) 

    dict_data_labels = dict_labels(path_doc)

    return np.array(data), np.array([dict_data_labels[title] 
                                     for title in titles])

def visualize_imgs(X, y, label, normalized=True):
    '''
    Returns an plot of the image

    Arguments:
        X: The data itself
        y: Label for each data points
        label: A condition that can be either 'positive' or 'negative'
        normalized: Boolean value to inform whether the image values has been normalized or not
    '''

    fig = plt.figure(figsize=(18, 18))
    columns = 4
    rows = 1
    X_filt = X[y == label]
    y_filt = y[y == label]

    for i in range(1, columns*rows + 1):
        img = X_filt[i,:,:]
        lb = y_filt[i]
        fig.add_subplot(rows, columns, i)
        if normalized is True:
            plt.imshow(img, cmap='gray', vmin=0, vmax=1)
        else:
            plt.imshow(img, cmap='gray', vmin=0, vmax=255)
        plt.title(lb)
        plt.axis('off')
    plt.show()

def to_onehot_categorical(y, class_names):
    '''
    Returns one hot encoded classes
    
    Arguments:
        y: List of labels. e.g: 0 for cats and 1 for dogs y = [0, 0, 0, 1, 1]
        class_names: Names for each distinct labels. e.g 0 = cats and 1 = dogs
    '''
    label_encoder = LabelEncoder() 
    vec_y = label_encoder.fit_transform(y)      
    return to_categorical(vec_y, num_classes=len(class_names))   

def label_translator(class_names, y_pred):
    '''
    Returns a list of the predicted results translated into the original class names

    Arguments:
        class_names: Names of the classes in the dataset
        y_pred: Predicted class from the model
    '''

    y_copy = copy.deepcopy(y_pred)

    masked = []
    for prediction in y_copy:
        prediction[np.argmax(prediction)] = 1
        masked.append((prediction >= 1).astype('int32'))

    class_names_hot = to_onehot_categorical(class_names, class_names)

    stringed_hot = [str(onehot.astype('int32')) for onehot in class_names_hot]
    labels_dict = dict(zip(stringed_hot, class_names))
     
    return [labels_dict[str(array)] for array in masked]

def confusion_matrix_plot(y_test, y_pred, class_names):
    '''
    Returns a plot with the confusion matrix

    Arguments:
        y_test: Actual labels from the dataset
        y_pred: Predicted labels from the model
        class_names: Names of the classes in the dataset
    '''

    y_test_predicted = label_translator(class_names, y_pred)
    conf_matrix = confusion_matrix(y_test, y_test_predicted)

    fig, ax = plt.subplots(figsize=(6, 6))
    ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
    for i in range(conf_matrix.shape[0]):
        for j in range(conf_matrix.shape[1]):
            ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', 
                    size='xx-large')
    
    plt.xlabel('Predictions', fontsize=14)
    plt.ylabel('Actuals', fontsize=14)
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=90)
    plt.yticks(tick_marks, class_names)
    plt.title('Confusion Matrix\n\n', fontsize=14)
    plt.show()

def random_positive(test_path, y_test, y_test_pred):
    '''
    Returns a path to a correctly classified as positive
    random file from the test set and its index

    Arguments:
        test_path: Path to the test dataset
        y_test: Actual labels from the dataset
        y_test_pred: Predicted labels from the model
    '''

    files = [test_path + '/' + file for file in os.listdir(test_path)]

    indexes = [index for index, element in enumerate(y_test_pred) if 
               element == 'positive' and y_test[index] == 'positive']

    image_index = random.choice(indexes)

    return files[image_index], image_index

## Data Pre-Processing

In [ ]:
train_path = 
train_test = 
train_path_doc = 
